### Here, I will experiment with new models

In [54]:
import torch
from torch import nn
from vit_pytorch.vit import Transformer
from einops.layers.torch import Rearrange
from einops import rearrange, repeat

In [77]:
class VitModel(nn.Module):
    
    def __init__(self, *, image_size, patch_size, 
                 dim, depth, heads, mlp_dim, num_classes=1000, pool = 'cls', channels = 3, 
                 dim_head = 64, dropout = 0., emb_dropout = 0.):
        super().__init__()
        
        assert image_size % patch_size == 0, 'Image dimensions must be divisible by the patch size.'
        num_patches = (image_size // patch_size) ** 2
        patch_dim = channels * patch_size ** 2
        assert pool in {'cls', 'mean'}, 'pool type must be either cls (cls token) or mean (mean pooling)'

        self.to_patch_embedding = nn.Sequential(
            Rearrange('b c (h p1) (w p2) -> b (h w) (p1 p2 c)', p1 = patch_size, p2 = patch_size),
            nn.Linear(patch_dim, dim),
        )

        self.pos_embedding = nn.Parameter(torch.randn(1, num_patches + 1, dim))
        self.cls_token = nn.Parameter(torch.randn(1, 1, dim))
        self.dropout = nn.Dropout(emb_dropout)

        self.transformer = Transformer(dim, depth, heads, dim_head, mlp_dim, dropout)

        self.pool = pool
        self.to_latent = nn.Identity()

        

        self.mlp_head = nn.Sequential(
            nn.LayerNorm(1024, elementwise_affine=True),
            nn.Linear(1024, 4))
        self._digit1 = nn.Sequential(
            nn.LayerNorm(1024, elementwise_affine=True),
            nn.Linear(1024, 11))
        self._digit2 = nn.Sequential(
            nn.LayerNorm(1024, elementwise_affine=True),
            nn.Linear(1024, 11))
        
    def forward(self, img):
        x = self.to_patch_embedding(img)
        b, n, _ = x.shape

        cls_tokens = repeat(self.cls_token, '() n d -> b n d', b = b)
        x = torch.cat((cls_tokens, x), dim=1)
        x += self.pos_embedding[:, :(n + 1)]
        x = self.dropout(x)

        x = self.transformer(x)

        x = x.mean(dim = 1) if self.pool == 'mean' else x[:, 0]

        x = self.to_latent(x)

        length_logits = self.mlp_head(x)
        digit1_logits = self._digit1(x)
        digit2_logits = self._digit2(x)

        return length_logits, digit1_logits, digit2_logits
    
jerseyVit = VitModel(image_size = 256,
                        patch_size = 32,
                        dim = 1024,
                        depth = 6,
                        heads = 16,
                        mlp_dim = 2048,
                        dropout = 0.1,
                        emb_dropout = 0.1)

img = torch.randn(1, 3, 256, 256)
preds = jerseyVit(img) # (1, 1000)


jerseyVit = VitModel(image_size = 64,
                        patch_size = 16,
                        dim = 1024,
                        depth = 6,
                        heads = 16,
                        mlp_dim = 2048,
                        dropout = 0.1,
                        emb_dropout = 0.1)

img = torch.randn(1, 3, 64, 64)
preds = jerseyVit(img)

In [78]:
preds[0].shape, preds[1].shape, preds[2].shape

(torch.Size([1, 4]), torch.Size([1, 11]), torch.Size([1, 11]))